<a href="https://colab.research.google.com/github/apriandito/pertamina-2/blob/main/04_xai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install dalex

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix,
    roc_curve
)

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import dalex as dx

In [ ]:
# 1. Load training data
train_url = (
    "https://raw.githubusercontent.com/apriandito/pertamina-2/"
    "refs/heads/main/data/bbm_fraud_train.csv"
)
df_train = pd.read_csv(train_url)
df_train.head()

,volume_liters,total_amount,hour,is_weekend,loyalty_member,customer_transaction_count,days_since_last_transaction,same_day_transactions,volume_deviation,amount_deviation,...,is_night_transaction,bbm_type_encoded,payment_method_encoded,day_of_week_encoded,customer_type_encoded,spbu_category_encoded,spbu_province_encoded,spbu_city_encoded,is_fraud,fraud_type
0,578.37,8.039343e+06,7,True,False,1,0.0,1,0.000000,0.000000,...,False,3,1,2,1,0,5,13,0,NaN
1,664.13,4.516084e+06,8,False,False,2,109.0,1,0.069022,0.280616,...,False,4,2,6,1,1,3,26,0,NaN
2,163.24,2.024176e+06,12,False,False,3,37.0,1,0.651628,0.583492,...,False,2,2,0,1,1,2,7,0,NaN
3,54.64,7.594557e+05,4,False,False,4,54.0,1,0.850340,0.801955,...,True,3,4,4,1,1,8,24,1,multiple_cards
4,357.87,4.437588e+06,2,True,False,5,8.0,1,0.015894,0.121926,...,True,2,1,2,1,2,8,24,0,NaN


In [ ]:
# 2. Siapkan fitur & target
X = df_train.drop(columns=["is_fraud", "fraud_type"])
y = df_train["is_fraud"]

In [ ]:
# 3. Split data (chronological, tanpa shuffle)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

In [ ]:
# 4. Scaling fitur
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Ubah kembali ke DataFrame agar Dalex dapat menampilkan nama kolom
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
X_test_scaled  = pd.DataFrame(X_test_scaled,  columns=X.columns, index=X_test.index)

In [ ]:
# 5. Build & fit ANN (MLP)
model = MLPClassifier(
    hidden_layer_sizes=(100,),
    max_iter=200,
    random_state=42
)
model.fit(X_train_scaled, y_train)

MLPClassifier(random_state=42)

In [ ]:
# 6. Predict & evaluasi
y_pred   = model.predict(X_test_scaled)
y_proba  = model.predict_proba(X_test_scaled)[:, 1]

cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

accuracy    = accuracy_score(y_test, y_pred)
precision   = precision_score(y_test, y_pred)
recall      = recall_score(y_test, y_pred)
specificity = tn / (tn + fp)
f1          = f1_score(y_test, y_pred)
roc_auc     = roc_auc_score(y_test, y_proba)

print("=== Evaluation on Test Set (ANN MLP) ===")
print(f"Accuracy   : {accuracy:.4f}")
print(f"Precision  : {precision:.4f}")
print(f"Recall     : {recall:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"F1-score   : {f1:.4f}")
print(f"ROC AUC    : {roc_auc:.4f}")

=== Evaluation on Test Set (ANN MLP) ===
Accuracy   : 0.9895
Precision  : 0.9319
Recall     : 0.8514
Specificity: 0.9967
F1-score   : 0.8898
ROC AUC    : 0.9880


In [ ]:
# 7. Plot ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_proba)
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=fpr, y=tpr,
    mode='lines',
    name=f'ANN (AUC = {roc_auc:.3f})',
    line=dict(width=2)
))
fig.add_trace(go.Scatter(
    x=[0,1], y=[0,1],
    mode='lines',
    name='Random Guess',
    line=dict(dash='dash', width=1)
))
fig.update_layout(
    title='ROC Curve - ANN (MLP) Fraud Detection',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    legend=dict(x=0.65, y=0.15),
    template='plotly_white',
    width=700,
    height=500
)
fig.show()

In [ ]:
# 8. Real‑time prediction
rt_url = (
    "https://raw.githubusercontent.com/apriandito/pertamina-2/"
    "refs/heads/main/data/bbm_fraud_realtime.csv"
)
df_rt = pd.read_csv(rt_url)
feature_cols = X.columns.tolist()

X_rt = df_rt[feature_cols]
X_rt_scaled = scaler.transform(X_rt)
X_rt_scaled = pd.DataFrame(X_rt_scaled, columns=feature_cols, index=df_rt.index)

df_rt['predicted_is_fraud'] = model.predict(X_rt_scaled)
df_rt['fraud_probability']  = model.predict_proba(X_rt_scaled)[:, 1]

print("\n=== Real‑time Prediction Summary (ANN MLP) ===")
print(f"Total transaksi    : {len(df_rt)}")
print(f"Predicted fraud     : {df_rt['predicted_is_fraud'].sum()} "
      f"({df_rt['predicted_is_fraud'].mean()*100:.2f}%)")
print(f"Average fraud prob  : {df_rt['fraud_probability'].mean():.4f}")


=== Real‑time Prediction Summary (ANN MLP) ===
Total transaksi    : 5000
Predicted fraud     : 461 (9.22%)
Average fraud prob  : 0.0956


In [ ]:
# 9. XAI dengan Dalex
explainer = dx.Explainer(model, X_train_scaled, y_train, label="ANN (MLP)")

Preparation of a new explainer is initiated

  -> data              : 40000 rows 21 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 40000 values
  -> model_class       : sklearn.neural_network._multilayer_perceptron.MLPClassifier (default)
  -> label             : ANN (MLP)
  -> predict function  : <function yhat_proba_default at 0x79b06caebba0> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 2.77e-23, mean = 0.049, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.987, mean = 2.03e-05, max = 1.0
  -> model_info        : package sklearn

A new explainer has been created!


In [ ]:
# 9.1 Model performance
mp = explainer.model_performance()
mp.plot()

In [ ]:
# 9.2 Feature importance (permutation)
parts = explainer.model_parts()
parts.plot()

In [ ]:
# 9.3 Partial dependence profiles
profile = explainer.model_profile(variables=X.columns.tolist())
profile.plot()

Calculating ceteris paribus: 100%|██████████| 21/21 [00:00<00:00, 34.58it/s]


In [ ]:
# 9.4 Breakdown plot untuk sample transaksi realtime pertama
sample_rt = X_rt_scaled.iloc[0]
bd_rt = explainer.predict_parts(sample_rt, type='break_down')
bd_rt.plot()